# Pro tip: if you run the import once you get a lot of warning, but then you just run it again then the warning is gone

In [2]:
import impnb
import herMLE
import Data_processing as dp;
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Import_EEG as IE;
img_path='/cndd/hop006/Processing/PRL_imgs'

# Part Zero: Gathering Data
#### Note: Only doing from rat_26 and up, some are not 20 days apart
#### Common electrodes: 
 Ag Disk (frontal center)',
 'L Au1 EEG',
 'L Cerebellum EEG',
 'L NAc LFP',
 'L lOFC LFP',
 'R ACC LFP',
 'R Au1 LFP',
 'R Caudate LFP',
 'R lOFC EEG'}

In [3]:
rat_2_anal=sorted(list(set(dp.day1['subject'])))[1:] # droped 25 cuz it only had one recording
myrat=rat_2_anal[1] # loop starts here   
val_chan,chan_name,dates,rat_nums,reward_tone=IE.get_excel_info(myrat)
days=2
rat_rlogs=[]
rat_alogs=[]
for day in range(days):
    rat_df=IE.get_ephys(dates[day],rat_nums[day])
    tvec,signal,low_tone,high_tone,tar,off_tar=IE.rat_summary(rat_df)
    action_log,reward_log,epoched_eeg=IE.get_behavior(rat_df,eeg_length=300,plot=False)
    rat_rlogs.append(reward_log)
    rat_alogs.append(action_log)


# Part One: Behavior Models
### Model-1: Q-learning with alpha_gain and loss, beta, and sigmoid action decision

In [6]:
alphaGs,alphaLs,betas=[],[],[]
for rlog,alog in zip(rat_rlogs,rat_alogs): #
    alphaG,alphaL,beta=herMLE.my_minimize(alog,rlog)
    print(alphaG,alphaL,beta)
    alphaGs.append(alphaG)
    alphaLs.append(alphaL)
    betas.append(beta)

0.05 0.9500000000000001 0.5
0.9500000000000001 0.9500000000000001 1.0


### Plots

# Part two: import EEG